In [50]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
from ipywidgets import widgets
from IPython.display import display, HTML
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns

# Customized modules
HOME = '/home/serinatan/project/GPU-Virtualization-Benchmarks/util'
if HOME not in sys.path:
    sys.path.append(HOME)

import data.scripts.common.format as fmt
import data.scripts.common.constants as const
import data.scripts.gen_tables.gen_pair_configs as gen_pair

SCRIPT_PATH = os.path.join(HOME, 'data/scripts') 
PKL_PATH = os.path.join(HOME, 'data/pickles') 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
def draw_table(df, cols, hide_index=True):
    if hide_index:
        return df[cols].style.set_table_styles(fmt.table_style).hide_index()
    else:
        return df[cols].style.set_table_styles(fmt.table_style)

# Seq

In [6]:
# run python scripts to generate all the pickles needed
gen_seq = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_seq.py')   
%run $gen_seq 

df_seq = pd.read_pickle(os.path.join(PKL_PATH, 'seq.pkl'))
print(df_seq.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'grid_x', 'grid_y',
       'grid_z', 'block_x', 'block_y', 'block_z', 'ctas/SM', 'runtime', 'ipc',
       'instructions', 'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
       'avg_mrq_latency', 'stall_core_ldst', 'stall_icnt_to_l2', 'l2_BW',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'stall_l2_to_icnt',
       'l1D_miss_rate', 'l2_miss_rate', 'l2_total_accesses', 'packet_lat_out',
       'network_lat_out', 'inject_out', 'accepted_out', 'packet_lat_in',
       'network_lat_in', 'inject_in', 'accepted_in',
       'mem_subpartition_parallism', 'mem_subpartition_parallism_util',
       'L2_reservation_fail', 'empty_warp', 'stall_warp', 'idle_warp',
       'scoreboard_warp', 'tot_warp_insn', 'regs', 'smem', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mrqq', 'total_cmd', 'wasted_col',
       'wasted_row', 'mem_idle', 'CCDLc', 'WTRc', 'RTWc', 'RCDc', 'RCDWRc',
       'avg_dram_bw', 'avg_dram_eff', 'std_dram_bw', 'r

In [56]:
col_seq = ['pair_str', 'config', 'runtime', 'instructions', 'avg_dram_bw', 
           'ratio_dram_bw', 'avg_dram_eff', 'avg_mem_lat', 'MPKI', 'waves',
           'l2_total_accesses', 'l2_access_density',
          ]

draw_table(df_seq, col_seq).format(
    {'avg_dram_bw':'{:.4f}', 
     'ratio_dram_bw': '{:.2f}',
     'MPKI': '{:.2f}',
     'waves': '{:.2f}',
     'l2_total_accesses': '{:,}',
     'l2_access_density': '{:.1f}',
     'instructions': '{:,}',
    })

pair_str,config,runtime,instructions,avg_dram_bw,ratio_dram_bw,avg_dram_eff,avg_mem_lat,MPKI,waves,l2_total_accesses,l2_access_density
cut_sgemm-0,TITANV-SEP_RW,93149,"228,392,960",0.0883,0.00,0.570446,298,1.26,0.80,"525,696",2.3
cut_sgemm-1,TITANV-SEP_RW,1719066,"6,580,076,544",0.0742,0.00,0.465546,931,0.34,3.20,"8,381,824",1.3
cut_wmma-0,TITANV-SEP_RW,62957,"36,372,480",0.0003,0.43,nan,819,1.81,0.40,"516,480",14.2
cut_wmma-1,TITANV-SEP_RW,2257808,"2,045,247,488",0.5092,0.01,0.706908,3280,9.58,3.20,"28,075,706",13.7
parb_cutcp-0,TITANV-SEP_RW,525176,"469,895,962",0.0000,0.72,nan,154,0.02,0.09,"185,724",0.4
parb_lbm-0,TITANV-SEP_RW,875934,"427,778,603",0.7181,0.05,0.742696,14605,26.30,18.75,"11,254,960",26.3
parb_spmv-0,TITANV-SEP_RW,73206,"52,720,795",0.7725,0.00,0.859517,795,18.51,0.90,"1,138,494",21.6
parb_stencil-0,TITANV-SEP_RW,350242,"644,994,210",0.7136,0.01,0.7437,1229,6.68,0.80,"5,576,344",8.6
rod_heartwall-0,TITANV-SEP_RW,10070,"4,044,300",0.0013,0.53,0.0940933,154,5.19,0.13,"43,701",10.8
rod_hotspot-0,TITANV-SEP_RW,29318,"89,989,280",0.0001,1.77,nan,460,0.48,2.89,"212,520",2.4


# Intra

In [9]:
gen_intra = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_intra.py')
%run $gen_intra

In [12]:
df_intra = pd.read_pickle(os.path.join(PKL_PATH, 'intra.pkl'))
col_intra = ['pair_str', 'intra', 'l2', 'norm_ipc']

# draw_table(df_intra, col_intra)

In [13]:
df_intra_best = pd.read_pickle(os.path.join(PKL_PATH, 'intra_best.pkl'))
df_intra_best.sort_values('pair_str', inplace=True)
col_intra_best = ['pair_str', 
        'perfdollar', 'intra', 'l2', 'norm_ipc', 
        'runtime',
        #'l2_miss_rate',
        #'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
        'comp_busy', 'dram_busy', 'avg_dram_bw', 'avg_dram_eff']

draw_table(df_intra_best, col_intra_best).format({'norm_ipc': "{:.4f}", 'dominant_resc': '{:.2f}'})

pair_str,perfdollar,intra,l2,norm_ipc,runtime,comp_busy,dram_busy,avg_dram_bw,avg_dram_eff
cut_sgemm-0,0.669156,2,0.5,0.9614,96892,0.259849,0.181625,0.129729,0.644896
cut_sgemm-1,1.43829,1,0.5,1.1162,1540155,0.460856,0.205192,0.113583,0.428508
cut_wmma-0,1.68589,2,0.5,0.9463,66533,0.0559222,0.0935402,0.0432762,0.299054
cut_wmma-1,0.886275,2,0.5,0.9722,2322385,0.0999374,0.733181,0.501917,0.667796
parb_cutcp-0,3.27202,2,0.5,1.0000,525176,0.127457,9.28537e-05,2.21786e-05,nan
parb_lbm-0,0.700971,3,0.5,0.9121,960397,0.0511992,0.980072,0.655588,0.667817
parb_spmv-0,0.725113,6,0.5,1.0264,71326,0.0759997,0.927519,0.814725,0.872437
parb_stencil-0,0.726218,3,0.5,0.9429,371444,0.221282,0.959338,0.6521,0.671542
rod_heartwall-0,3.17514,1,0.5,1.0000,10070,0.0646486,0.00632534,0.00137254,0.09346
rod_hotspot-0,2.36684,2,0.5,0.9012,32531,0.329445,0.118075,0.0683838,0.322858


# Algorithm: Find pair configs

In [61]:
find_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_pair_configs.py')
col_prod = ['norm_ipc_x', 'norm_ipc_y', 'diff_mflat', 'sum_ipc', 
            'intra_x', 'intra_y', 'l2_x', 'l2_y',
           'sum_comp', 'sum_dram', 'penalized']


app1 = widgets.Dropdown(
    options=df_seq['pair_str'],
    value='cut_sgemm-0',
    rows=5,
    description='App 1:',
    disabled=False
)

app2 = widgets.Dropdown(
    options=df_seq['pair_str'],
    value='cut_sgemm-0',
    rows=5,
    description='App 2:',
    disabled=False
)

qos = widgets.FloatSlider(
    value=0.75,
    min=0.1,
    max=0.95,
    step=0.05,
    description='QoS:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

button = widgets.Button(description='Calculate', button_style='info')

def show_widgets():
    display(app1)
    display(app2)
    display(qos)
    display(button)
    
show_widgets()

def onclick(b):
    clear_output()
    show_widgets()
    %run $find_pair --app $app1.value $app2.value --qos $qos.value
    
    df_prod = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
   
    display(HTML(draw_table(df_prod, col_prod, False).render()))
    
    
button.on_click(onclick)

Dropdown(description='App 1:', index=6, options=('cut_sgemm-0', 'cut_sgemm-1', 'cut_wmma-0', 'cut_wmma-1', 'pa…

Dropdown(description='App 2:', index=10, options=('cut_sgemm-0', 'cut_sgemm-1', 'cut_wmma-0', 'cut_wmma-1', 'p…

FloatSlider(value=0.75, continuous_update=False, description='QoS:', max=0.95, min=0.1, step=0.05)

Button(button_style='info', description='Calculate', style=ButtonStyle())

,norm_ipc_x,norm_ipc_y,diff_mflat,sum_ipc,intra_x,intra_y,l2_x,l2_y,sum_comp,sum_dram,penalized
0,0.755625,0.793606,17,1.54923,3,2,0.75,0.25,0.166934,1.84892,rod_hotspot3d-0
1,0.750882,0.832365,36,1.58325,3,2,0.5,0.5,0.17229,1.84615,rod_hotspot3d-0
2,1.02636,0.958086,46,1.98444,6,4,0.5,0.5,0.212504,1.83899,parb_spmv-0
3,0.976302,0.920045,56,1.89635,6,4,0.25,0.75,0.204022,1.84384,parb_spmv-0
4,1.04622,0.793606,124,1.83982,6,2,0.75,0.25,0.18935,1.84706,rod_hotspot3d-0
5,1.02636,0.832365,152,1.85872,6,2,0.5,0.5,0.19328,1.84133,rod_hotspot3d-0
6,0.750882,0.958086,162,1.70897,3,4,0.5,0.5,0.191515,1.8438,parb_spmv-0
7,1.04622,0.794549,168,1.84077,6,4,0.75,0.25,0.192039,1.84024,parb_spmv-0
8,0.976302,0.850014,178,1.82632,6,2,0.25,0.75,0.19266,1.84046,rod_hotspot3d-0
9,0.755625,0.794549,275,1.55017,3,4,0.75,0.25,0.169623,1.84209,parb_spmv-0


In [33]:
list_pairs = []
qos = 0.75

for bench1 in const.app_for_pair:
    for bench2 in const.app_for_pair:
        if bench1 < bench2:
            config_str = gen_pair.main(['--app', bench1, bench2])
            if len(config_str) > 0: 
                df_first = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
                list_pairs.append(df_first.head(1).reset_index(drop=True))
            

In [47]:
df_pairs = pd.concat(list_pairs, axis=0).reset_index()
df_pairs.sort_values(['sum_dram', 'sum_comp'], ascending=False, inplace=True)
draw_table(df_pairs, ['pair_str_x', 'pair_str_y'] + col_prod, False)
            

,pair_str_x,pair_str_y,norm_ipc_x,norm_ipc_y,diff_mflat,sum_ipc,intra_x,intra_y,l2_x,l2_y,sum_comp,sum_dram,penalized
44,parb_spmv-0,parb_stencil-0,1.02636,0.908696,4,1.93505,6,6,0.5,0.5,0.289238,1.88759,parb_spmv-0
40,parb_lbm-0,rod_hotspot3d-0,0.912054,0.916045,1542,1.8281,3,6,0.5,0.5,0.184366,1.88646,rod_hotspot3d-0
41,parb_lbm-0,rod_streamcluster-0,0.91658,0.921749,1616,1.83833,3,2,0.75,0.25,0.130187,1.87774,rod_streamcluster-0
37,parb_lbm-0,parb_spmv-0,0.91658,0.931435,1474,1.84801,3,12,0.75,0.25,0.122806,1.8751,parb_spmv-0
32,parb_stencil-0,rod_hotspot3d-0,0.789056,0.850014,15,1.63907,3,2,0.25,0.75,0.304747,1.8636,rod_hotspot3d-0
36,parb_lbm-0,parb_stencil-0,0.920605,0.776432,1234,1.69704,3,12,0.25,0.75,0.263544,1.86234,parb_stencil-0
60,rod_hotspot3d-0,rod_streamcluster-0,0.793606,0.810678,89,1.60428,2,1,0.25,0.75,0.195601,1.84983,rod_hotspot3d-0
47,parb_spmv-0,rod_hotspot3d-0,0.755625,0.793606,17,1.54923,3,2,0.75,0.25,0.166934,1.84892,rod_hotspot3d-0
33,parb_stencil-0,rod_streamcluster-0,0.798044,0.994988,11,1.79303,6,2,0.25,0.75,0.278356,1.83236,parb_stencil-0
48,parb_spmv-0,rod_streamcluster-0,1.04622,0.751851,0,1.79807,6,1,0.75,0.25,0.154063,1.80145,rod_streamcluster-0


# Pair w.r.t. Seq

In [37]:
gen_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_pair.py')
baseline = 'seq'
baseline_pkl = os.path.join(PKL_PATH, 'seq.pkl')
output = os.path.join(PKL_PATH, 'pair_seq.pkl')

%run $gen_pair --output $output --baseline_pkl $baseline_pkl --baseline $baseline

In [39]:
df_pair_seq = pd.read_pickle(os.path.join(PKL_PATH, 'pair_seq.pkl'))
df_pair_seq.sort_values(['1_bench', '2_bench'], inplace=True)
col_pair_seq = ['1_bench', '2_bench', '1_sld', '2_sld', 'ws', 'fairness', '1_norm_mflat', '2_norm_mflat']
draw_table(df_pair_seq, col_pair_seq)

1_bench,2_bench,1_sld,2_sld,ws,fairness,1_norm_mflat,2_norm_mflat
cut_sgemm-1,parb_lbm-0,0.277375,0.893025,1.1704,0.310601,2.77658,0.152687
cut_sgemm-1,parb_spmv-0,0.962535,0.864206,1.82674,0.897844,0.900107,0.622642
cut_sgemm-1,parb_stencil-0,0.730872,0.761517,1.49239,0.959759,0.513426,0.228641
cut_wmma-0,parb_lbm-0,0.24173,0.849939,1.09167,0.284408,3.22955,0.178843
cut_wmma-0,parb_spmv-0,0.671119,0.508979,1.1801,0.758404,1.82173,0.906918
cut_wmma-0,parb_stencil-0,0.798147,0.440257,1.2384,0.551599,1.55067,0.481692
parb_cutcp-0,parb_lbm-0,0.648061,0.890846,1.53891,0.727467,11.8442,0.154878
parb_cutcp-0,parb_spmv-0,0.959922,1.00107,1.96099,0.958899,1.38961,0.543396
parb_cutcp-0,parb_stencil-0,0.938066,0.923632,1.8617,0.984613,1.15584,0.202604


# Pair w.r.t Intra Best

In [40]:
gen_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_pair.py')
baseline = 'intra'
baseline_pkl = os.path.join(PKL_PATH, 'intra_best.pkl')
output = os.path.join(PKL_PATH, 'pair_intra.pkl')

%run $gen_pair --output $output --baseline_pkl $baseline_pkl --baseline $baseline

In [41]:
df_pair_intra = pd.read_pickle(os.path.join(PKL_PATH, 'pair_intra.pkl'))
df_pair_intra.sort_values(['1_bench', '2_bench'], inplace=True)

col_pair_intra = ['1_bench', '2_bench', '1_sld', '2_sld', 'ws', 'fairness', '1_norm_mflat', '2_norm_mflat']
draw_table(df_pair_intra, col_pair_intra)

1_bench,2_bench,1_sld,2_sld,ws,fairness,1_norm_mflat,2_norm_mflat
cut_sgemm-1,parb_lbm-0,0.248507,0.979136,1.22764,0.253802,4.20325,0.979359
cut_sgemm-1,parb_spmv-0,0.862359,0.842012,1.70437,0.976405,1.3626,1.16471
cut_sgemm-1,parb_stencil-0,0.654807,0.807615,1.46242,0.810791,0.777236,1.11952
cut_wmma-0,parb_lbm-0,0.25546,0.931895,1.18735,0.274129,2.60079,1.14712
cut_wmma-0,parb_spmv-0,0.709239,0.495908,1.20515,0.699212,1.46706,1.69647
cut_wmma-0,parb_stencil-0,0.843482,0.466908,1.31039,0.553548,1.24877,2.35857
parb_cutcp-0,parb_lbm-0,0.648061,0.976747,1.62481,0.663489,11.8442,0.993412
parb_cutcp-0,parb_spmv-0,0.959922,0.975358,1.93528,0.984173,1.38961,1.01647
parb_cutcp-0,parb_stencil-0,0.938066,0.979544,1.91761,0.957656,1.15584,0.992032


# SMK w.r.t Seq

In [42]:
gen_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_pair.py')
baseline = 'seq'
csv = os.path.join(HOME, 'data/csv/pair_smk.csv')
baseline_pkl = os.path.join(PKL_PATH, 'seq.pkl')
output = os.path.join(PKL_PATH, 'smk.pkl')

%run $gen_pair --csv $csv --output $output --baseline_pkl $baseline_pkl --baseline $baseline --smk

In [43]:
df_smk = pd.read_pickle(os.path.join(PKL_PATH, 'smk.pkl'))
df_smk.sort_values(['1_bench', '2_bench'], inplace=True)

col_smk = ['1_bench', '2_bench', '1_sld', '2_sld', 'ws', 'fairness', '1_norm_mflat', '2_norm_mflat']
draw_table(df_smk, col_smk)

1_bench,2_bench,1_sld,2_sld,ws,fairness,1_norm_mflat,2_norm_mflat
cut_sgemm-1,parb_lbm-0,0.180838,0.950202,1.13104,0.190316,7.11493,0.362547
cut_sgemm-1,parb_spmv-0,0.954379,0.948842,1.90322,0.994198,1.00967,0.738365
cut_sgemm-1,parb_stencil-0,0.663187,0.910176,1.57336,0.728636,0.996778,0.430431
cut_wmma-0,parb_lbm-0,0.124711,0.891608,1.01632,0.139872,7.17216,0.408079
cut_wmma-0,parb_spmv-0,0.624722,0.607166,1.23189,0.971898,1.91941,1
cut_wmma-0,parb_stencil-0,0.491898,0.671498,1.1634,0.732538,2.2149,0.713588
parb_cutcp-0,parb_lbm-0,0.309201,0.987677,1.29688,0.313059,67.3312,0.898802
parb_cutcp-0,parb_spmv-0,0.879459,0.992489,1.87195,0.886115,3.54545,0.992453
parb_cutcp-0,parb_stencil-0,0.802363,0.996123,1.79849,0.805485,6.04545,0.8869
